# selenium

In [1]:
import json
import re

In [2]:
from selenium.webdriver.common.keys import Keys
import selenium.webdriver


In [5]:
driver = selenium.webdriver.Chrome('/Users/vicky/Dev/chromedriver')
driver.get("https://www.quora.com/search?q=founder&type=profile")

In [6]:
driver.window_handles

['CDwindow-D7310C62AE0A3B6EA2ED5D76243C79AC']

In [7]:
def get_company(company_string):

    for index, word in enumerate(company_string):
        if word in ['Founder', 'Co-founder', 'CEO', 'founder'] :
            company = ' '.join(company_string[index:])
            
            if '.' in company:
                stop_pos = [m.start(0) for m in re.finditer(r'\.', company)][0]
            else:
                stop_pos = len(company)  
            
            return company[:stop_pos]


In [8]:
def get_name_company(query_list):
    sample_db = []
    for name in query_list:
        name_com = name.replace('\n', ' ').split(', ')
        if len(name_com) > 1:
            sample_db.append({name_com[0]: get_company(name_com[1].split(" "))})
            
    return sample_db

# infinite scrolling

In [9]:
import time

In [10]:
# driver.find_element_by_link_text("All").click()
for i in range(1,20):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)


In [11]:
html = driver.page_source
print(html.encode('utf-8'))

b'<html lang="en" class="js-wf-loaded"><head><link rel="icon" href="https://qsf.fs.quoracdn.net/-3-images.favicon.ico-26-ae77b637b1e7ed2c.ico"><script src="https://apis.google.com/_/scs/apps-static/_/js/k=oz.gapi.en_US.ar2HFI7WvyM.O/m=auth2/exm=client/rt=j/sv=1/d=1/ed=1/am=wQE/rs=AGLTcCMoUc2VjSUz_OwZhlxgC3a8IMPILg/cb=gapi.loaded_1" async=""></script><script src="https://connect.facebook.net/en_US/sdk.js?hash=64e8b4f7937393bb73270c922db6ef14&amp;ua=modern_es6" async="" crossorigin="anonymous"></script><script src="https://apis.google.com/_/scs/apps-static/_/js/k=oz.gapi.en_US.ar2HFI7WvyM.O/m=client/rt=j/sv=1/d=1/ed=1/am=wQE/rs=AGLTcCMoUc2VjSUz_OwZhlxgC3a8IMPILg/cb=gapi.loaded_0" async=""></script><script async="" src="//www.google-analytics.com/analytics.js"></script><script type="text/javascript" async="" src="//apis.google.com/js/client:platform.js?onload=gapiAsyncInit" gapi_processed="true"></script><script type="text/javascript" async="" src="//connect.facebook.net/en_US/sdk.js"></s

In [12]:
element = driver.find_elements_by_class_name("QueryResultsList")
print(element[0].text)

Profiles for founder
Clear Filters
 Profile:
Charlie Cheever, Quora
In addition to being a founder at Quora: Palo Alto, Facebook, Amazon, Seattle, Harvard, Shady Side Academy, Pittsburgh… (more)
Profile:
Mark Ferguson, Founder InvestFourMore, Real estate Investor and Broker at Blue Steel Real Estate
17 year real estate investor, real estate agent, best-selling author, and car lover. I have flipped more than 155 houses, and bought 20 rentals in my career. I have 19 flip...(more)
Profile:
Dave Consiglio, Chemistry and Physics High School Teacher and Community College Professor
™ … Co-Author of: A Comedy of (Classroom) Errors … CEO and Founder of: Consiglio Devastations...… (more)
Profile:
Josh Fechter, Author | Advisor | Entrepreneur
I’m the co-founder of a writing software, Squibler. … I’m also the founder of Product & Growth, a community of 25,000 members. … Want my latest book on scaling growth… (more)
Profile:
Karen Marie Shelton, B.A.& M.A. & PostGrad in Sociology, Public Policy & B

In [13]:
query_list = element[0].text.split("Profile:")
print(query_list)

['Profiles for founder\nClear Filters\n ', '\nCharlie Cheever, Quora\nIn addition to being a founder at Quora: Palo Alto, Facebook, Amazon, Seattle, Harvard, Shady Side Academy, Pittsburgh… (more)\n', '\nMark Ferguson, Founder InvestFourMore, Real estate Investor and Broker at Blue Steel Real Estate\n17 year real estate investor, real estate agent, best-selling author, and car lover. I have flipped more than 155 houses, and bought 20 rentals in my career. I have 19 flip...(more)\n', '\nDave Consiglio, Chemistry and Physics High School Teacher and Community College Professor\n™ … Co-Author of: A Comedy of (Classroom) Errors … CEO and Founder of: Consiglio Devastations...… (more)\n', '\nJosh Fechter, Author | Advisor | Entrepreneur\nI’m the co-founder of a writing software, Squibler. … I’m also the founder of Product & Growth, a community of 25,000 members. … Want my latest book on scaling growth… (more)\n', "\nKaren Marie Shelton, B.A.& M.A. & PostGrad in Sociology, Public Policy & Busi

In [14]:
get_name_company(query_list)

[{' Charlie Cheever': 'founder at Quora: Palo Alto'},
 {' Mark Ferguson': 'Founder InvestFourMore'},
 {' Dave Consiglio': 'CEO and Founder of: Consiglio Devastations'},
 {' Josh Fechter': None},
 {' Karen Marie Shelton': None},
 {' Oliver Emberton': 'founder of Silktide Entrepreneur'},
 {' Kyle Murao': None},
 {' Rand Fishkin': 'Founder of SparkToro'},
 {' Jason M. Lemkin': None},
 {' Gayle Laakmann McDowell': None},
 {' Pat Roberts': None},
 {' Dharmesh Shah': 'Founder and CTO'},
 {' Vaibhav Mallya': None},
 {' Soleio': 'founder & partner of Combine'},
 {' John DeMarchi': None},
 {' Bram Cohen': 'founder of BitTorrent Inc'},
 {' Mircea Goia': 'Founder of RomanianStartups'},
 {' Eric Mayefsky': None},
 {' Anne K. Halsall': None}]

# send to bq

In [15]:
from google.cloud import bigquery
client = bigquery.Client.from_service_account_json(
    "/Users/vicky/Dev/seraph_etl.json"
)


In [ ]:
schema = [
    bigquery.SchemaField("founder", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("company", "STRING", mode="Nullable"),
]

table_id = "pure-silicon-196123.seraph_v1.company_quora"

table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)  # API request
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

In [2]:
# driver.quit()

NameError: name 'driver' is not defined